In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import os,sys
import vptq
import transformers
# os.environ["CUDA_VISIBLE_DEVICES"]="7"
model_name = "VPTQ-community/Qwen2.5-7B-Instruct-v8-k65536-0-woft"  # 替换为你需要的模型名称

tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=True)
model = vptq.AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,  # 使用半精度加速推理
        trust_remote_code=True,
).eval()

prompt="hello"
out = model.generate(
    **tokenizer(prompt, return_tensors="pt").to("cuda"),
    max_new_tokens=100,
    pad_token_id=2
)

# 

Successfully loaded VPTQ CUDA kernels.


Replacing linear layers...: 100%|██████████| 399/399 [00:00<00:00, 3622.63it/s]
/home/jzgrp/xiaomenghan/anaconda3/envs/pytorch310/lib/python3.10/site-packages/accelerate/utils/modeling.py:1517: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issu

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (k_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (v_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (o_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (up_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (down_proj): VQuantLinear(
            (centroids): Embedding(1, 524288)
          )
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm(

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
path="/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_indices.pt"
matrix = torch.load(path)  # 若文件在GPU上，需先转到CPU: matrix = torch.load('matrix.pt').cpu()
q_centroids=model.model.layers[0].self_attn.q_proj.centroids
q_cen_weight=q_centroids.weight.view(1,65536,8)
# 2. 转换为NumPy数组
# np_matrix = matrix.cpu().numpy()
np_matrix=q_cen_weight.cpu().detach().numpy()
# 3. 检查维度并调整（假设需要二维）
print("矩阵形状:", np_matrix.shape)
if np_matrix.ndim > 2:
    # 示例：如果维度是3D（如通道x高度x宽度），取第一个通道
    np_matrix = np_matrix[0][0:512]
elif np_matrix.ndim == 1:
    raise ValueError("矩阵是一维的，无法直接可视化。")

# 4. 使用热图可视化
plt.figure(figsize=(8, 32))
plt.imshow(np_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='indices')
plt.title("Indices data distribution")
plt.xlabel("column index")
plt.ylabel("row index")

# 5. 保存图片
plt.savefig('indices_data.png')


In [ ]:
q_centroids=model.model.layers[0].self_attn.q_proj.centroids
q_cen_weight=q_centroids.weight.view(1,65536,8)
print(q_cen_weight.shape)
print(matrix.shape,matrix.dtype)
a=matrix[0,0,0]
# a=a&0x0000ffff
print(a)
a=torch.bitwise_and(a, 0xFFFF).to(torch.uint16)
print(a)
print(q_cen_weight[0,a,:].shape)

In [ ]:
high_mask=0xffff0000
low_mask=0x0000ffff
new_index=torch.zeros([448,3584])
for j in range(matrix.shape[1]):
    for k in range(matrix.shape[2]):
        packint32=matrix[0,j,k].to(torch.int32)
        new_index[j,2*k+1]=torch.bitwise_right_shift(packint32.to(torch.int64),16).to(torch.uint16)
        new_index[j,2*k]=torch.bitwise_and(packint32, 0xFFFF).to(torch.uint16)




In [ ]:
channel_vector=torch.zeros([5,448,8])
for i in range(channel_vector.shape[0]):
    for j in range(channel_vector.shape[1]):
        index=new_index[j,i].to(torch.uint16)
        channel_vector[i,j,:]=q_cen_weight[0,index,:]
    

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(8, 32))
channel_vector=channel_vector.cpu().detach().numpy()
for i in range(5):
    # 绘制每个子图
    im = axs[i].imshow(channel_vector[i,:,:], cmap='viridis', interpolation='nearest')
    axs[i].set_title(f"Indices data distribution {i+1}")
    axs[i].set_xlabel("column index")
    axs[i].set_ylabel("row index")
    
    # 添加颜色条
    cbar = fig.colorbar(im, ax=axs[i], orientation='vertical', pad=0.05, aspect=40)
    cbar.set_label('indices')

# 调整子图之间的间距
plt.tight_layout()

# 显示图表
plt.show()
